In [1]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, learning_curve,validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error,cohen_kappa_score
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from scipy.stats import norm, chi2
from mlxtend.evaluate import paired_ttest_5x2cv

In [2]:
#import data
df_metrics = pd.read_csv('Data/metrics_corr.csv')

#remove columns that used kl evaluation metric
kl_list = [col for col in df_metrics.columns if col.endswith('_kl')]
df_metrics.drop(kl_list, axis=1, inplace=True)

In [3]:
#create age categories
df_metrics.loc[(df_metrics['DoB']>1971) & (df_metrics['DoB']<=1983), 'age_class'] = 0
df_metrics.loc[(df_metrics['DoB']>1983) & (df_metrics['DoB']<=1994), 'age_class'] = 1
df_metrics.loc[(df_metrics['DoB']>1994) & (df_metrics['DoB']<=2000), 'age_class'] = 2
df_metrics.loc[(df_metrics['DoB']>2000) & (df_metrics['DoB']<=2011), 'age_class'] = 3
df_metrics['age_class'] = pd.to_numeric(df_metrics['age_class'], downcast = 'integer')
df_metrics = df_metrics[df_metrics.age_class.isnull() == False]

In [4]:
#create gender category
le = LabelEncoder()
df_metrics['age_var'] = le.fit_transform(df_metrics['age_class'])
df_metrics['gender_var'] = le.fit_transform(df_metrics['Gender'])

In [6]:
#create lists that store column names for each saliency map

aim_list = [col for col in df_metrics.columns if col.startswith('aim_')]
rare_list = [col for col in df_metrics.columns if col.startswith('rare_')]
qss_list = [col for col in df_metrics.columns if col.startswith('qss_')]
lds_list = [col for col in df_metrics.columns if col.startswith('lds_')]
imsig_list = [col for col in df_metrics.columns if col.startswith('imsig_')]
ikn_list = [col for col in df_metrics.columns if col.startswith('ikn_')]
gbvs_list = [col for col in df_metrics.columns if col.startswith('gbvs_')]
gaus_list = [col for col in df_metrics.columns if col.startswith('gauss_')]
fes_list = [col for col in df_metrics.columns if col.startswith('fes_')]
dva_list = [col for col in df_metrics.columns if col.startswith('dva_')]
cvs_list = [col for col in df_metrics.columns if col.startswith('cvs_')]
cas_list = [col for col in df_metrics.columns if col.startswith('cas_')]
dgi_list = [col for col in df_metrics.columns if col.startswith('dgi_')]
dgii_list = [col for col in df_metrics.columns if col.startswith('dgii_')]
dgiie_list = [col for col in df_metrics.columns if col.startswith('dgiie_')]
icf_list = [col for col in df_metrics.columns if col.startswith('icf_')]
all_pred = icf_list + dgiie_list + dgii_list + dgi_list + cas_list + cvs_list + dva_list + fes_list + gbvs_list + ikn_list + imsig_list + lds_list + qss_list + rare_list + aim_list
d_aim_list = [col for col in df_metrics.columns if col.startswith('d_aim_')]

In [8]:
#import selected columns for age and sex
df_age_cols = pd.read_csv('Data/age_selectk.csv')
df_sex_cols = pd.read_csv('Data/sex_selectk.csv')

In [9]:
#create y variables
ya = df_metrics['age_class']
yg = df_metrics['gender_var']

In [76]:
#get column names of the top 50 most relevant variables
temp = df_age_cols.nlargest(50,'fs_score')
cols = temp['col_name'].tolist()

## Classify age data

In [11]:
#create logistric regression model
lreg = LogisticRegression(max_iter=2000)

In [21]:
#split data into train and test data and scale
X_train, X_test, y_train, y_test = train_test_split(df_metrics[cols], ya, test_size=0.2,random_state=131) 
y_train = le.fit_transform(y_train)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [197]:
#run sequential forward feature selection to pick number fo variables
sfs2 = sfs(lreg, k_features=50, forward=True, verbose=2, scoring='accuracy')
sfs2 = sfs2.fit(X_train, y_train);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.2s finished

[2022-06-27 00:12:21] Features: 1/50 -- score: 0.30440492476060194[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:    3.5s finished

[2022-06-27 00:12:25] Features: 2/50 -- score: 0.306812585499316[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    5.2s finished

[2022-06-27 00:12:30] Features: 3/50 -- score: 0.2952393980848153[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:   11.6s finished

[2022-06-27 00:17:33] Features: 28/50 -- score: 0.3299589603283174[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:   11.1s finished

[2022-06-27 00:17:44] Features: 29/50 -- score: 0.33004103967168263[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    9.5s finished

[2022-06-27 00:17:53] Features: 30/50 -- score: 0.327688098495212[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    6.9s finished

[2022-06-27 00:18:01] Features: 31/

In [198]:
#8 variables selected 
sfs1 = sfs(lreg, k_features=8, forward=True, verbose=2, scoring='accuracy')
sfs1 = sfs1.fit(X_train, y_train);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.2s finished

[2022-06-27 00:19:00] Features: 1/8 -- score: 0.30440492476060194[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:    1.8s finished

[2022-06-27 00:19:01] Features: 2/8 -- score: 0.306812585499316[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    2.5s finished

[2022-06-27 00:19:04] Features: 3/8 -- score: 0.2952393980848153[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

In [199]:
#get feature names of 8 selected variables
feat_names = list(sfs1.k_feature_names_)
print(feat_names)

['3', '4', '12', '16', '17', '20', '34', '49']


In [77]:
feat_names = ['3', '4', '12', '16', '17', '20', '34', '49']
feat_num_age = [3,4,12,16,17,20,34,49]

In [23]:
#create x_train and x_test selecting only the 8 relevant variables
X_age = np.take(X_train, feat_names, axis = 1)
X_test = np.take(X_test, feat_names, axis = 1)

In [80]:
#get the column names of relevant variables
col_name = temp.iloc[feat_num_age, 1].tolist()
col_name

['qss_auc',
 'qss_sauc',
 'cvs_nss',
 'cvs_cc',
 'gbvs_sim',
 'qss_sim',
 'rare_ig',
 'imsig_sim']

## hyperparameter tuning

In [73]:
#define parameter ranges
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l2','l1', 'elasticnet']
c_values = [100, 10, 1.0, 0.1, 0.01, 0.001, 0.0001]

In [202]:
#create model and tune parameters using gridsearch
model = LogisticRegression(max_iter=2000,class_weight='balanced')
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=15, n_repeats=10, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_age, y_train)
#output best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.342660 using {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}


C:\Users\njeri\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
8400 fits failed out of a total of 15750.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1050 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\njeri\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    execution. Reducing this number can be useful to avoid an
  File "C:\Users\njeri\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    Parameters
  File "C:\Users\njeri\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError:

## age model hypertuned & features selected

In [203]:
#create model with tuned parameter, fit and calculate accuracy
model_age = LogisticRegression(penalty = 'l2', solver = 'liblinear', C = 1, max_iter = 1000,class_weight='balanced')
model_age.fit(X_age,y_train)
pred = model_age.predict(X_test)
print(classification_report(y_test, pred))
print("Model F1 Score ", f1_score(y_test, pred, average = 'micro'))

              precision    recall  f1-score   support

         0.0       0.29      0.43      0.35        21
         1.0       0.35      0.36      0.36        33
         2.0       0.48      0.45      0.47        31
         3.0       0.54      0.32      0.40        22

    accuracy                           0.39       107
   macro avg       0.42      0.39      0.39       107
weighted avg       0.42      0.39      0.40       107

Model F1 Score  0.39252336448598124


In [24]:
#create model with tuned parameter, fit and calculate accuracy - kfold cross validation to verify baseline accuracy

model_age = LogisticRegression(penalty = 'l2', solver = 'liblinear', C = 1, max_iter = 1000,class_weight='balanced')
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(model_age, X_age, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
model_age.fit(X_age, y_train)
pred = model_age.predict(X_test)
print(classification_report(y_test, pred))
print("Model F1 Score ", f1_score(y_test, pred, average = 'micro'))

Baseline: 34.67% (2.59%)
              precision    recall  f1-score   support

         0.0       0.22      0.33      0.26        21
         1.0       0.38      0.42      0.40        33
         2.0       0.50      0.29      0.37        31
         3.0       0.40      0.36      0.38        22

    accuracy                           0.36       107
   macro avg       0.37      0.35      0.35       107
weighted avg       0.39      0.36      0.36       107

Model F1 Score  0.35514018691588783


# SEX

In [28]:
#select top 50 variables relevant in predicting sex
temp = df_sex_cols.nlargest(50,'fs_score')
cols_sex = temp['col_name'].tolist()

In [109]:
#split data into train and test and scale
X_train, X_test, y_train, y_test = train_test_split(df_metrics[cols_sex], yg, test_size=0.2,random_state=131) 
y_train = le.fit_transform(y_train)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [154]:
#select features using sequential forward feature selection
sfs3 = sfs(lreg, k_features=50, forward=True, verbose=2, scoring='accuracy')
sfs3 = sfs3.fit(X_train, y_train);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.7s finished

[2022-06-26 23:37:27] Features: 1/50 -- score: 0.5877701778385773[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:    2.4s finished

[2022-06-26 23:37:30] Features: 2/50 -- score: 0.5971545827633379[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    2.7s finished

[2022-06-26 23:37:32] Features: 3/50 -- score: 0.6042407660738714[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    5.6s finished

[2022-06-26 23:39:43] Features: 28/50 -- score: 0.6087277701778386[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    7.8s finished

[2022-06-26 23:39:51] Features: 29/50 -- score: 0.6016963064295486[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    5.0s finished

[2022-06-26 23:39:56] Features: 30/50 -- score: 0.5946648426812586[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    4.5s finished

[2022-06-26 23:40:01] Features: 31/

In [172]:
#7 features selected
sfs4 = sfs(lreg, k_features=7, forward=True, verbose=2, scoring='accuracy')
sfs4 = sfs4.fit(X_train, y_train);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.0s finished

[2022-06-26 23:48:27] Features: 1/7 -- score: 0.5877701778385773[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:    2.3s finished

[2022-06-26 23:48:30] Features: 2/7 -- score: 0.5971545827633379[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    2.8s finished

[2022-06-26 23:48:33] Features: 3/7 -- score: 0.6042407660738714[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

In [173]:
#get feature names of 7 relevant features
feat_names = list(sfs4.k_feature_names_)
print(feat_names)

['0', '3', '20', '32', '33', '36', '38']


In [106]:
feat_names = ['0', '3', '20', '32', '33', '36', '38']
feat_num = [0, 3, 20, 32, 33, 36, 38]

In [110]:
#filter train and test to only 7 relevant variables
X_gender = np.take(X_train, feat_names, axis = 1)
X_test = np.take(X_test, feat_names, axis = 1)

In [185]:
#set parameters for hypertuning
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l2','l1', 'elasticnet']
c_values = [100, 10, 1.0, 0.1, 0.01, 0.001, 0.0001]

In [186]:
#create model and run gridsearch to tune parameters
model = LogisticRegression(max_iter=2000,class_weight='balanced')
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=15, n_repeats=10, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_gender, y_train)

#output selected parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.571568 using {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}


C:\Users\njeri\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
8400 fits failed out of a total of 15750.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1050 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\njeri\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    execution. Reducing this number can be useful to avoid an
  File "C:\Users\njeri\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    Parameters
  File "C:\Users\njeri\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError:

## sex model hypertuned & features selected

In [111]:
#create model with tuned parameters and calculate accuracy
model_sex = LogisticRegression(penalty = 'l2', solver = 'liblinear', C = 0.1, max_iter = 1000,class_weight='balanced')
model_sex.fit(X_gender,y_train)
pred = model_sex.predict(X_test)
print(classification_report(y_test, pred))
print("Model F1 Score ", f1_score(y_test, pred, average = 'micro'))

              precision    recall  f1-score   support

           0       0.53      0.66      0.59        53
           1       0.56      0.43      0.48        54

    accuracy                           0.54       107
   macro avg       0.55      0.54      0.54       107
weighted avg       0.55      0.54      0.54       107

Model F1 Score  0.5420560747663551


In [20]:
#run kfold cross validation to verify baseline training accuracy
model_sex = LogisticRegression(penalty = 'l2', solver = 'liblinear', C = 0.1, max_iter = 1000,class_weight='balanced')
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(model_sex, X_gender, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
model_sex.fit(X_gender, y_train)
pred = model_sex.predict(X_test)
print(classification_report(y_test, pred))
print("Model F1 Score ", f1_score(y_test, pred, average = 'micro'))

Baseline: 55.49% (9.37%)
              precision    recall  f1-score   support

           0       0.53      0.66      0.59        53
           1       0.56      0.43      0.48        54

    accuracy                           0.54       107
   macro avg       0.55      0.54      0.54       107
weighted avg       0.55      0.54      0.54       107

Model F1 Score  0.5420560747663551


In [119]:
#get column names of features used in model
col_name_sex = temp.iloc[feat_num, 1].tolist()

In [120]:
col_name_sex

['icf_cc', 'qss_auc', 'qss_sim', 'dva_ig', 'rare_nss', 'dva_sim', 'cvs_auc']

In [121]:
#check coefficients of sex model
model_sex.coef_[0]

array([ 0.30412581,  0.14802472, -0.33759704,  0.03597867, -0.00416017,
        0.0553091 , -0.10738364])

In [122]:
#create data frame with coefficients and input variable names
sex_coef = pd.DataFrame([col_name,model_sex.coef_[0]])
sex_coef

,0,1,2,3,4,5,6
0,icf_cc,qss_auc,qss_sim,dva_ig,rare_nss,dva_sim,cvs_auc
1,0.304126,0.148025,-0.337597,0.035979,-0.00416,0.055309,-0.107384


In [84]:
#baseline classifier - predict dominant class for age
X_train, X_test, y_train, y_test = train_test_split(df_metrics[cols], df_metrics.age_class, test_size=0.2,random_state=131)
dummy_clf_age = DummyClassifier(strategy='most_frequent')
dummy_clf_age.fit(X_test, y_test)
dummy_clf_age.score(X_test, y_test)

0.308411214953271

In [ ]:
#5x2 cv t test to compare age model to baseline classifier
y = df_metrics['age_class']
x = df_metrics[col_name]

t, p = paired_ttest_5x2cv(estimator1=model,
                          estimator2=dummy_clf_age,
                          X=x, y=y,
                          random_seed=1)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

In [ ]:
#baseline classifier - predict dominant class for gender
X_train, X_test, y_train, y_test = train_test_split(df_metrics[cols], df_metrics.gender_var, test_size=0.2,random_state=131)
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_test, y_test)
dummy_clf.score(X_test, y_test)

In [125]:
#5x2 cv t test to compare mo
y = df_metrics['gender_var']
x = df_metrics[col_name_sex]

t, p = paired_ttest_5x2cv(estimator1=model_sex,
                          estimator2=dummy_clf,
                          X=x, y=y,
                          random_seed=1)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

t statistic: 0.368
p value: 0.728
